In [48]:
# To be implemented
# 1- Multiple Decision Makers
# 2- Fuzzy Classification (Pharma/Non)
# 3- 

In [49]:
import pandas as pd

df = pd.read_csv('diabetes2.csv')
df.head()
small_constant = 1e-6 

# Replace zero values with the small constant in the remaining columns
df[df.columns] = df[df.columns].replace(0, small_constant)
df.head()


,Pregnancies MAX,Glucose MAX,BloodPressure MAX,SkinThickness MIN,Insulin MIN,BMI MAX,Age MAX
0,Low,Low,Medium,Low,High,High,Low
1,High,High,High,Medium,Medium,High,High
2,Medium,Medium,High,High,Low,Low,Low
3,Low,Low,Low,High,High,Low,Low


In [50]:
replace_dict = {
    'Low': '1,3,5',
    'Medium': '3,5,7',
    'High': '5,7,9'
}

# Replace the values in the DataFrame
df.replace(replace_dict, inplace=True)

# Add a 'Criteria' column
df['Criteria'] = df.index

# Move the 'Criteria' column to the front
df = df[['Criteria'] + [col for col in df.columns if col != 'Criteria']]

# Transpose the DataFrame
df = df.set_index('Criteria').T.reset_index()

df.head()



Criteria,index,0,1,2,3
0,Pregnancies MAX,"1,3,5","5,7,9","3,5,7","1,3,5"
1,Glucose MAX,"1,3,5","5,7,9","3,5,7","1,3,5"
2,BloodPressure MAX,"3,5,7","5,7,9","5,7,9","1,3,5"
3,SkinThickness MIN,"1,3,5","3,5,7","5,7,9","5,7,9"
4,Insulin MIN,"5,7,9","3,5,7","1,3,5","5,7,9"


In [22]:
# # Define a function to find the inverse of a string of numbers separated by commas and round to 2 decimal places
# def find_inverse(value):
#     numbers = value.split(',')
#     inverse_numbers = [format(1 / float(num), '.2f') for num in numbers]
#     return ','.join(inverse_numbers)

# # Apply the function to columns with "MIN" in their names
# min_columns = [col for col in df.columns if 'MIN' in col]
# df[min_columns] = df[min_columns].applymap(find_inverse)

# # Print the modified DataFrame
# print(df)
# df.head()

In [23]:
# # Define functions for normalization
# def normalize_max(column):
#     values = column.split(',')
#     max_value = max(map(int, values))
#     return [round(int(val) / max_value, 3) for val in values]

# def normalize_min(column):
#     values = column.split(',')
#     min_value = min(map(float, values))
#     normalized_values = [round(min_value / float(val), 3) for val in values]
#     return normalized_values

# # Apply normalization based on column names
# for column_name in df.columns:
#     if 'MAX' in column_name:
#         df[column_name] = df[column_name].apply(normalize_max)
#     elif 'MIN' in column_name:
#         df[column_name] = df[column_name].apply(normalize_min)

# print(df)

In [24]:
import skcriteria as skc 


In [28]:
# Remove the outer square brackets from all elements
df = df.applymap(lambda x: x[0])
matrix  = df.values.tolist()
# Remove the first column
matrix = [row[1:] for row in matrix]
matrix


[['1', '5', '3', '1'],
 ['1', '5', '3', '1'],
 ['3', '5', '5', '1'],
 ['1', '3', '5', '5'],
 ['5', '3', '1', '5'],
 ['5', '5', '1', '1'],
 ['1', '5', '1', '1']]

In [29]:
# Let's say we want to rank on which patient is more vulnerable to diabetes
# We set maximization for - pregnancy, glucose, bp, bmi and age
# We set minimization for skin thickness, insulin

objectives = [max, max, max, min]


# Assign weights to the conditions ( add up to 1 )

weights=[0.1, 0.2, 0.2, 0.15]

In [31]:
# Create the Decision Matrix

dm = skc.mkdm(
    matrix,
    objectives,
    weights,
    criteria= ["P1", "P2", "P3", "P4"]
)
dm

,P1[▲ 0.10],P2[▲ 0.20],P3[▲ 0.20],P4[▼ 0.15]
A0,1,5,3,1
A1,1,5,3,1
A2,3,5,5,1
A3,1,3,5,5
A4,5,3,1,5
A5,5,5,1,1
A6,1,5,1,1


In [32]:
dm.matrix  # this data ignores the objectives and the weights

Criteria,P1,P2,P3,P4
Alternatives,,,,
A0,1,5,3,1
A1,1,5,3,1
A2,3,5,5,1
A3,1,3,5,5
A4,5,3,1,5
A5,5,5,1,1
A6,1,5,1,1


In [33]:
#dm.plot.wheatmap()

In [34]:
type(dm)

skcriteria.core.data.DecisionMatrix

In [35]:
# Sum Scaler - Normalize scale (0-1)
from skcriteria.preprocessing import invert_objectives, scalers

scaler = scalers.SumScaler(target="weights") #normalize WEIGHTS only. Values normalized earlier.
dmt = scaler.transform(dm)
dmt

,P1[▲ 0.153846],P2[▲ 0.307692],P3[▲ 0.307692],P4[▼ 0.230769]
A0,1,5,3,1
A1,1,5,3,1
A2,3,5,5,1
A3,1,3,5,5
A4,5,3,1,5
A5,5,5,1,1
A6,1,5,1,1


In [37]:

# import matplotlib.pyplot as plt

# # Create a figure for the criteria KDE plot
# fig1, axs1 = plt.subplots(figsize=(6, 5))

# # Plot the criteria KDE
# dm.plot.kde(ax=axs1)
# axs1.set_title("Criteria KDE")

# # Show the figure
# plt.show()



In [39]:
# import matplotlib.pyplot as plt

# # Create a figure for the weights as bars plot
# fig2, axs2 = plt.subplots(figsize=(6, 5))

# # Plot the weights as bars
# dmt.plot.wbar(ax=axs2)
# axs2.set_title("Weights as Bars")

# # Show the figure
# plt.show()


In [40]:
# TOPSIS
from skcriteria.madm import similarity  
from skcriteria.pipeline import mkpipe  


In [41]:
pipe = mkpipe(
    invert_objectives.NegateMinimize(),
    scalers.VectorScaler(target="matrix"),  # this scaler transform the matrix
    scalers.SumScaler(target="weights"),  # and this transform the weights
    similarity.TOPSIS(),
)

pipe

<SKCPipeline [steps=[('negateminimize', <NegateMinimize []>), ('vectorscaler', <VectorScaler [target='matrix']>), ('sumscaler', <SumScaler [target='weights']>), ('topsis', <TOPSIS [metric='euclidean']>)]]>

In [42]:
rankTOPSIS = pipe.evaluate(dm)
rankTOPSIS

Alternatives,A0,A1,A2,A3,A4,A5,A6
Rank,2,2,1,4,6,3,5


In [45]:
import pandas as pd

rank_list = rankTOPSIS.rank_
rank_alternatives = rankTOPSIS.alternatives

rank_df_topsis = pd.DataFrame({
    'Rank': rank_list,
    'Feature': rank_alternatives
})

# Sort the DataFrame by rank in ascending order
rank_df_topsis = rank_df_topsis.sort_values(by='Rank', ascending=True)
rank_df_topsis = rank_df_topsis.reset_index(drop=True)

print(rank_df_topsis)


   Rank Feature
0     1      A2
1     2      A0
2     2      A1
3     3      A5
4     4      A3
5     5      A6
6     6      A4


In [46]:
#TOPSIS Metrics

print(rankTOPSIS.e_)
print("Ideal:", rankTOPSIS.e_.ideal)
print("Anti-Ideal:", rankTOPSIS.e_.anti_ideal)
print("Similarity index:", rankTOPSIS.e_.similarity)

<extra {'similarity', 'ideal', 'anti_ideal'}>
Ideal: [ 0.09691397  0.12865262  0.18258179 -0.03111692]
Anti-Ideal: [ 0.01938279  0.07719157  0.03651636 -0.15558458]
Similarity index: [0.58990409 0.58990409 0.83927806 0.4845058  0.28069066 0.5154942
 0.44887508]


In [262]:
#from skcriteria.preprocessing import invert_objectives, scalers

# Invert minimize - 1/ criterion

#inverter = invert_objectives.InvertMinimize()
#dmt = inverter.transform(dm)

In [47]:
# Weighted Sum Model
from skcriteria.madm import simple

dec = simple.WeightedSumModel()
rankWS = dec.evaluate(dm)  # we use the tansformed version of the data
rankWS

import pandas as pd

rank_list = rankWS.rank_
rank_alternatives = rankWS.alternatives

rank_df_ws = pd.DataFrame({
    'Rank': rank_list,
    'Candidate': rank_alternatives
})

# Sort the DataFrame by rank in ascending order
rank_df_ws = rank_df_ws.sort_values(by='Rank', ascending=True)
rank_df_ws = rank_df_ws.reset_index(drop=True)

print(rank_df_ws)

ValueError: WeightedSumModel can't operate with minimize objective